In [8]:
import requests
from itertools import chain
from collections import namedtuple
import logging

from lxml import etree
from pprint import pprint
from IPython.core.display import display, HTML
from copy import deepcopy

from io import StringIO

In [9]:
url = f"https://api.github.com/repos/BSData/wh40k/contents/"
files = requests.get(url).json()
catalogues = []
# 
for file in files:
    if file['type'] == 'file' and file['name'][0] != '.':
        catalogues.append(file['name'])

In [10]:
with open('src/battlescribe/game_schema.xsd', 'rb') as fh:
    game_schema = etree.XMLSchema(etree.fromstring(fh.read()))
with open('src/battlescribe/catelogue_schema.xsd', 'rb') as fh:
    catelogue_schema = etree.XMLSchema(etree.fromstring(fh.read()))

In [11]:
path = 'https://raw.githubusercontent.com/BSData/wh40k/master/{}'.format('Warhammer 40,000.gst')
r = requests.get(path)
base = etree.fromstring(
    r.content, 
    etree.XMLParser(
        remove_blank_text=True, schema = game_schema
    )
)

In [12]:
for catalogue in catalogues:
    name, ext = catalogue.split(".")
    if ext == 'cat':
        print(name)
        path = 'https://raw.githubusercontent.com/BSData/wh40k/master/{}'.format(
        catalogue)
        r = requests.get(path)
        cat = etree.fromstring(
            r.content, 
            etree.XMLParser(
                remove_blank_text=True, schema = catelogue_schema
            )
        )
        base_copy = deepcopy(base)
        for x in cat:
            base_copy.append(x)
        with open("src/battlescribe/catelogue.xslt", "r") as f:
            xslt_root = etree.XML(f.read())
        transform = etree.XSLT(xslt_root)
        result_tree = transform(base_copy)
        with open("src/battlescribe/{}.html".format(name), "w") as f:
            f.write(str(result_tree))
        #HTML(str(result_tree))

Aeldari - Craftworlds
Aeldari - Drukhari
Aeldari - FW Corsairs
Aeldari - Harlequins
Aeldari - Ynnari
Chaos - Chaos Knights
Chaos - Chaos Space Marines
Chaos - Daemons
Chaos - Dark Mechanicum
Chaos - Death Guard
Chaos - FW Heretic Astartes
Chaos - FW Renegade and Heretics
Chaos - Gellerpox Infected
Chaos - Servants of the Abyss
Chaos - Thousand Sons
Chaos - Titanicus Traitoris
Fallen
Imperium - Adepta Sororitas
Imperium - Adeptus Astra Telepathica
Imperium - Adeptus Custodes
Imperium - Adeptus Mechanicus
Imperium - Astra Militarum - Library
Imperium - Astra Militarum
Imperium - Black Templars
Imperium - Blackstone Fortress
Imperium - Blood Angels
Imperium - Dark Angels
Imperium - Deathwatch
Imperium - Elucidian Starstriders
Imperium - FW Adeptus Astartes
Imperium - FW Death Korps of Krieg
Imperium - FW Elysians
Imperium - Grey Knights
Imperium - Imperial Fists
Imperium - Imperial Knights
Imperium - Inquisition
Imperium - Iron Hands
Imperium - Legion of the Damned
Imperium - Officio Assa

In [13]:
for y in base.xpath(
    "//cat:*[@name = 'Storm Guardians']", 
    namespaces={
        'gc': 'http://www.battlescribe.net/schema/gameSystemSchema',
        'cat': 'http://www.battlescribe.net/schema/catalogueSchema'}):
    for line in str(etree.tostring(y, pretty_print=True)).split('\\n'):
        print(line)
    print()

In [11]:
# group profiles within an entry
# group entries by category HQ, ELite etec...